# Chapter 8 - Exercises

### 1. Modify the depth-first search function to produce a topological sort.

In [8]:
from pythonds3.graphs import Graph

class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def topological_sort(self):
        self.dfs()

        return sorted([vertex for vertex in self.get_vertices()], lambda vertex: vertex.get_closing_time(), reverse=True)

    def dfs(self):
        for vertex in self:
            vertex.color = "white"
            vertex.previous = -1
        for vertex in self:
            if vertex.color == "white":
                self.dfs_visit(vertex)

    def dfs_visit(self, start_vertex):
        start_vertex.color = "gray"
        self.time = self.time + 1
        start_vertex.discovery_time = self.time
        for next_vertex in start_vertex.get_neighbors():
            if next_vertex.color == "white":
                next_vertex.previous = start_vertex
                self.dfs_visit(next_vertex)
        start_vertex.color = "black"
        self.time = self.time + 1
        start_vertex.closing_time = self.time

### 2. Modify the depth-first search to produce strongly connected components.

We will need to compute the transpose of a graph with the following method.

In [9]:
from typing import Dict, List

def transpose(g: Dict[str, List]) -> Dict[str, List]:
    """
    Function used to transpose an input graph 'g'. 
    
    Input graph g is a dict where each key is the vertex v and each value
    an array of vertices w for which there is a directed edge
    from v to w.

    Example g = {
        "a": ["c", "d"],
        "b": ["d", "a"],
        "c": [],
        "d": ["c"]
    }
    """
    transposed_g = {v: [] for v in g.keys()}

    for v, adj_v in g.items():
        for w in adj_v:
            if v not in transposed_g[w]:
                transposed_g[w] += [v]

    return transposed_g

d1 = {"a": ["b", "c"], "b": ["c"], "c": []}

print(d1)
print(transpose({}))

{'a': ['b', 'c'], 'b': ['c'], 'c': []}
{}


In [10]:
from pythonds3 import Stack

class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0
        self.components = []

    def dfs(self):
        for vertex in self:
            vertex.color = "white"
            vertex.previous = -1
        for vertex in self:
            if vertex.color == "white":
                self.dfs_visit(vertex)

    def transpose(self):
        transposed_g = DFSGraph()

        for vertex in self:
            for neighbor in vertex.get_neighbors():
                if neighbor.get_neighbor(vertex) is None:
                    transposed_g.add_edge(neighbor, vertex, vertex.get_neighbor(neighbor))

        return transposed_g


    def dfs_visit(self, start_vertex=None):
        if start_vertex is None:
            start_vertex = self.get_vertices()[0]
        
        start_vertex.color = "gray"
        self.time = self.time + 1
        start_vertex.discovery_time = self.time
        for next_vertex in start_vertex.get_neighbors():
            if next_vertex.color == "white":
                next_vertex.previous = start_vertex
                self.dfs_visit(next_vertex)
        start_vertex.color = "black"
        self.time = self.time + 1
        start_vertex.closing_time = self.time

    
    def dfs_scc_visit(self, start_vertex=None, current_component=[]):
        if start_vertex is None:
            start_vertex = self.get_vertices()[0]

        start_vertex.color = "gray"

        if start_vertex.get_key() not in current_component:
            current_component.append(start_vertex.get_key())

        for next_vertex in sorted(start_vertex.get_neighbors(), key=lambda x: x.closing_time, reverse=True):
            if next_vertex.color == "white":
                next_vertex.previous = start_vertex
                self.dfs_scc_visit(next_vertex, current_component)
        
        start_vertex.color = "black"

        self.components += [current_component]

    
    def scc_algorithm(self, start_vertex=None):
        if start_vertex is None:
            start_vertex = self.get_vertices()[0]

        self.dfs_visit(start_vertex)
        transposed_g = self.transpose()
        transposed_g.dfs_scc_visit()

        

### 3. Write the `transpose` method for the `Graph` class.

See above.

### 4. Using breadth-first search write an algorithm that can determine the shortest path from each vertex to every other vertex. This is called the “all pairs shortest path problem.”

### 5. Using breadth-first search write an algorithm that can determine the shortest path from each vertex to every other vertex. This is called the “all pairs shortest path problem.”

In [17]:
from sys import maxsize
from itertools import product
from pythonds3 import Graph

import random


def generate_random_graph(n_vertices : int, threshold : float = 0.5, min_weight : int = -5, max_weight : int = 5, no_loops : bool = True):
    random_graph = Graph()

    for i in range(n_vertices):
        random_graph.set_vertex(str(i))

    for i in range(n_vertices):
        for j in range(n_vertices):
            if no_loops and i == j:
                continue
            if random.random() < threshold:
                random_graph.add_edge(str(i), str(j), random.randint(min_weight, max_weight))
            if random.random() < threshold:
                random_graph.add_edge(str(j), str(i), random.randint(min_weight, max_weight))

    return random_graph


def get_distance(graph : Graph, a, b):
    """
    Helper to find distance between two vertices identified by their keys a and b
    """
    if a not in graph.get_vertices():
        raise IndexError(f"Vertex {a} not found")
    if b not in graph.get_vertices():
        raise IndexError(f"Vertex {b} not found")

    return graph.get_vertex(a).get_neighbor(graph.get_vertex(b))


def all_pairs_shortest_path(graph: Graph):
    """
    Shortest path from vertex "a" to vertex "b":
        - graph: Graph
    """
    all_vertices_couples = list(product([v for v in graph.get_vertices()], [v for v in graph.get_vertices()]))
    distances = {(x, y): (maxsize if x != y else 0) for (x, y) in all_vertices_couples}
    routes = {(x, y): [] for (x, y) in all_vertices_couples}
    
    for (v, w) in distances:
        if (v, w) in graph.get_edges():
            distances[(v, w)] = get_distance(graph, v, w)
            routes[(v, w)] += [(v, w)]
    
    to_visit = list(graph.get_vertices())

    for vertex in to_visit:
        # BFS to find shorted path from vertex to everyone else
        visit_queue = [vertex]
        visited = []
        
        while visit_queue:
            current_vertex = visit_queue.pop()
            for neighbor in [n.get_key() for n in graph.get_vertex(current_vertex).get_neighbors()]:
                if neighbor not in visited:
                    distance_through_current = distances[(vertex, current_vertex)] + distances[(current_vertex, neighbor)]
                    route_through_current = routes[(vertex, current_vertex)] + routes[(current_vertex, neighbor)]
                    if distance_through_current < distances[(vertex, neighbor)]:
                        distances[(vertex, neighbor)] = distance_through_current
                        routes[(vertex, neighbor)] = route_through_current
                    visit_queue.append(neighbor)
                visited.append(current_vertex)

    return {couple: {"distance": distances[couple], "route": routes[couple]} for couple in distances}



g = generate_random_graph(5, 0.50, 0, 10)
all_pairs_shortest_path(g)


{('0', '0'): {'distance': 0, 'route': []},
 ('0', '1'): {'distance': 8, 'route': [('0', '3'), ('3', '1')]},
 ('0', '2'): {'distance': 1, 'route': [('0', '2')]},
 ('0', '3'): {'distance': 0, 'route': [('0', '3')]},
 ('0', '4'): {'distance': 9, 'route': [('0', '4')]},
 ('1', '0'): {'distance': 0, 'route': [('1', '0')]},
 ('1', '1'): {'distance': 0, 'route': []},
 ('1', '2'): {'distance': 1, 'route': [('1', '0'), ('0', '2')]},
 ('1', '3'): {'distance': 0, 'route': [('1', '0'), ('0', '3')]},
 ('1', '4'): {'distance': 7, 'route': [('1', '4')]},
 ('2', '0'): {'distance': 0, 'route': [('2', '0')]},
 ('2', '1'): {'distance': 15, 'route': [('2', '3'), ('3', '1')]},
 ('2', '2'): {'distance': 0, 'route': []},
 ('2', '3'): {'distance': 7, 'route': [('2', '3')]},
 ('2', '4'): {'distance': 1, 'route': [('2', '4')]},
 ('3', '0'): {'distance': 5, 'route': [('3', '0')]},
 ('3', '1'): {'distance': 6, 'route': [('3', '2'), ('2', '4'), ('4', '1')]},
 ('3', '2'): {'distance': 5, 'route': [('3', '2')]},
 ('

### 6. Using breadth-first search revise the maze program from the Chapter 4 (Recursion) to find the shortest path out of a maze.

In [18]:
import turtle

START = "S"
OBSTACLE = "+"
TRIED = "."
DEAD_END = "-"
PART_OF_PATH = "O"


class Maze:
    def __init__(self, maze_filename):
        with open(maze_filename, "r") as maze_file:
            self.maze_list = [
                [ch for ch in line.rstrip("\n")]
                for line in maze_file.readlines()
            ]
        self.rows_in_maze = len(self.maze_list)
        self.columns_in_maze = len(self.maze_list[0])
        for row_idx, row in enumerate(self.maze_list):
            if START in row:
                self.start_row = row_idx
                self.start_col = row.index(START)
                break

        self.x_translate = -self.columns_in_maze / 2
        self.y_translate = self.rows_in_maze / 2
        # self.t = turtle.Turtle()
        # self.t.shape("turtle")
        # self.wn = turtle.Screen()
        # self.wn.setworldcoordinates(
        #     -(self.columns_in_maze - 1) / 2 - 0.5,
        #     -(self.rows_in_maze - 1) / 2 - 0.5,
        #     (self.columns_in_maze - 1) / 2 + 0.5,
        #     (self.rows_in_maze - 1) / 2 + 0.5,
        # )

    def draw_maze(self):
        self.t.speed(10)
        self.wn.tracer(0)
        for y in range(self.rows_in_maze):
            for x in range(self.columns_in_maze):
                if self.maze_list[y][x] == OBSTACLE:
                    self.draw_centered_box(
                        x + self.x_translate, -y + self.y_translate, "orange"
                    )
        self.t.color("black")
        self.t.fillcolor("blue")
        self.wn.update()
        self.wn.tracer(1)

    def draw_centered_box(self, x, y, color):
        self.t.up()
        self.t.goto(x - 0.5, y - 0.5)
        self.t.color(color)
        self.t.fillcolor(color)
        self.t.setheading(90)
        self.t.down()
        self.t.begin_fill()
        for i in range(4):
            self.t.forward(1)
            self.t.right(90)
        self.t.end_fill()

    def move_turtle(self, x, y):
        self.t.up()
        self.t.setheading(self.t.towards(x + self.x_translate, -y + self.y_translate))
        self.t.goto(x + self.x_translate, -y + self.y_translate)

    def drop_bread_crumb(self, color):
        self.t.dot(10, color)

    def update_position(self, row, col, val=None):
        if val:
            self.maze_list[row][col] = val
        self.move_turtle(col, row)

        if val == PART_OF_PATH:
            color = "green"
        elif val == OBSTACLE:
            color = "red"
        elif val == TRIED:
            color = "black"
        elif val == DEAD_END:
            color = "red"
        else:
            color = None

        if color:
            self.drop_bread_crumb(color)

    def is_exit(self, row, col):
        return (
            row == 0
            or row == self.rows_in_maze - 1
            or col == 0
            or col == self.columns_in_maze - 1
        )

    def __getitem__(self, idx):
        return self.maze_list[idx]


def search_from(maze, start_row, start_column):
# try each of four directions from this point until we find a way out.
# base Case return values:
#  1. We have run into an obstacle, return false
    maze.update_position(start_row, start_column)
    if maze[start_row][start_column] == OBSTACLE:
        return False
    #  2. We have found a square that has already been explored
    if (
        maze[start_row][start_column] == TRIED
        or maze[start_row][start_column] == DEAD_END
    ):
        return False
    # 3. We have found an outside edge not occupied by an obstacle
    if maze.is_exit(start_row, start_column):
        maze.update_position(start_row, start_column, PART_OF_PATH)
        return True
    maze.update_position(start_row, start_column, TRIED)
    # Otherwise, use logical short circuiting to try each direction
    # in turn (if needed)
    found = (
        search_from(maze, start_row - 1, start_column)
        or search_from(maze, start_row + 1, start_column)
        or search_from(maze, start_row, start_column - 1)
        or search_from(maze, start_row, start_column + 1)
    )
    if found:
        maze.update_position(start_row, start_column, PART_OF_PATH)
    else:
        maze.update_position(start_row, start_column, DEAD_END)
    return found


my_maze = Maze("assets/maze2.txt")
# my_maze.draw_maze()
# my_maze.update_position(my_maze.start_row, my_maze.start_col)

# search_from(my_maze, my_maze.start_row, my_maze.start_col)


In [44]:
def is_valid_block(matrix : list[list[str]], i, j):
    return i >= 0 and j >= 0 and i < len(matrix) and j < len(matrix[i])

def maze_to_graph(maze : Maze) -> Graph:
    graph_maze = Graph() 

    for i, _ in enumerate(maze.maze_list):
        for j, val in enumerate(maze.maze_list[i]):
            if val != OBSTACLE:
                graph_maze.set_vertex((i, j))
                
    for i, _ in enumerate(maze.maze_list):
        for j, val in enumerate(maze.maze_list[i]):
            if val != OBSTACLE:
                for adj_i in [i - 1, i + 1]:
                    if is_valid_block(maze.maze_list, adj_i, j) and maze.maze_list[adj_i][j] != OBSTACLE:
                        graph_maze.add_edge((i, j), (adj_i, j), 1)
                for adj_j in [j - 1, j + 1]:
                    if is_valid_block(maze.maze_list, i, adj_j) and maze.maze_list[i][adj_j] != OBSTACLE:
                        graph_maze.add_edge((i, j), (i, adj_j), 1)

    return graph_maze

def move_turtle_to_exit(maze : Maze, start_row : int, start_column : int):
    pass

def shortest_path_out(maze : Maze, start_row : int, start_column : int):
    exit_coordinates = [(0, j) for j in range(maze.columns_in_maze) if maze.maze_list[0][j] != OBSTACLE] 
    exit_coordinates += [(i, 0) for i in range(maze.rows_in_maze) if maze.maze_list[i][0] != OBSTACLE]
    exit_coordinates += [(maze.rows_in_maze - 1, j) for j in range(maze.columns_in_maze) if maze.maze_list[maze.rows_in_maze - 1][j] != OBSTACLE]
    exit_coordinates += [(i, maze.columns_in_maze - 1) for i in range(maze.rows_in_maze) if maze.maze_list[i][maze.columns_in_maze - 1] != OBSTACLE]
    
    graph_maze = maze_to_graph(maze)
    d = all_pairs_shortest_path(graph_maze)
    distances = {((start_row, start_column), (i, j)): d[((start_row, start_column), (i, j))]["distance"] for (i, j) in exit_coordinates}
    routes = {((start_row, start_column), (i, j)): d[((start_row, start_column), (i, j))]["route"] for (i, j) in exit_coordinates}
    
    return {couple: {"distance": distances[couple], "route": routes[couple]} for couple in distances}

graph_maze = maze_to_graph(my_maze)

print(shortest_path_out(my_maze, 6, 5))

{((6, 5), (2, 0)): {'distance': 9, 'route': [((6, 5), (5, 5)), ((5, 5), (5, 4)), ((5, 4), (5, 3)), ((5, 3), (5, 2)), ((5, 2), (5, 1)), ((5, 1), (4, 1)), ((4, 1), (3, 1)), ((3, 1), (2, 1)), ((2, 1), (2, 0))]}}


### 7. Write a program to solve the following problem: you have two jugs, a 4-gallon and a 3-gallon. Neither of the jugs has any markings. There is a pump that can be used to fill the jugs with water. How can you get exactly two gallons of water in the 4-gallon jug?

In [77]:
def fill_jug(jugs : tuple[int, int], which_jug : int = None):
    match which_jug:
        case 0:
            return (4, jugs[1])
        case 1:
            return (jugs[0], 3)
        case _:
            return (4, 3)
        
def empty_jug(jugs : tuple[int, int], which_jug : int = None):
    match which_jug:
        case 0:
            return (0, jugs[1])
        case 1:
            return (jugs[0], 0)
        case _:
            return (0, 0)
        
def pour_jug(jugs : tuple[int, int], from_jug : int, to_jug : int):
    if from_jug == to_jug:
        raise IndexError("from_jug and to_jug cannot be the same!")

    amount_available_from = jugs[from_jug]

    if amount_available_from == 0:
        return jugs
    else:
        new_jugs = list(jugs)
        match to_jug:
            case 0:
                amount_available_to = 4 - jugs[to_jug]
            case 1:
                amount_available_to = 3 - jugs[to_jug]
            case _:
                raise IndexError("Incorrect to_jug!")
            
    
        amount_to_pour = amount_available_to if amount_available_from > amount_available_to else amount_available_from
        
        new_jugs[from_jug] = jugs[from_jug] - amount_to_pour
        new_jugs[to_jug] = jugs[to_jug] + amount_to_pour

        return tuple(new_jugs)

def combinations_graph(jugs : tuple[int, int] = (0, 0), all_states : list[tuple[int, int]] = [], starting_state : tuple[int, int] = (0, 0)):
    state_graph = Graph()
    to_visit = [starting_state]
    visited = []

    for state in all_states:
        state_graph.set_vertex(state)

    while to_visit:
        state = to_visit.pop(0)
        if state not in visited:
            all_reachable_states = [fill_jug(state, 0)] + [fill_jug(state, 1)] + [fill_jug(state)] + [pour_jug(state, jug1, jug2) for (jug1, jug2) in [(0, 1), (1, 0)]] + [empty_jug(state, 0)] + [empty_jug(state, 1)] + [empty_jug(state)]
            new_states = [new_state for new_state in all_reachable_states if new_state in all_states]
            to_visit.extend(new_states)
            for new_state in new_states:
                state_graph.add_edge(state, new_state, 1)
            visited.append(state)

    return state_graph

def get_filling_sequence(jugs, starting_state = (0, 0), final_state = (0, 2)):
    all_states = [(i, j) for i in range(5) for j in range(4)]
    state_graph = combinations_graph(jugs, all_states, starting_state)
    all_pairs = all_pairs_shortest_path(state_graph)
    return all_pairs[(starting_state, final_state)]

get_filling_sequence((0, 0), (0, 0), (0, 2))


{'distance': 5,
 'route': [((0, 0), (0, 3)),
  ((0, 3), (3, 0)),
  ((3, 0), (3, 3)),
  ((3, 3), (4, 2)),
  ((4, 2), (0, 2))]}

### 8. Generalize the problem above so that the parameters to your solution include the size of each jug and the final amount of water to be left in the larger jug.

In [173]:
class JugProblemSolver:
    def __init__(self) -> None:
        pass

    def set_jugs_number(self, number_of_jugs):
        self._number_of_jugs = number_of_jugs
        self._jugs = tuple(0 for _ in range(self._number_of_jugs))
        return self
    
    def set_jugs_sizes(self, jugs_sizes : tuple[int]):
        if type(jugs_sizes) is not tuple[int] and len(jugs_sizes) != self._number_of_jugs:
            raise Exception(f"Jugs sizes format is wrong: {jugs_sizes} but requered (x, y, z, ...) format!")
        self._sizes = jugs_sizes
        possible_states_iterable = (range(size + 1) for size in self._sizes)
        self._all_possible_states = list(product(*possible_states_iterable))
        return self

    def set_initial_state(self, initial_state : tuple[int] = None):
        if type(initial_state) is not tuple[int] and len(initial_state) != self._number_of_jugs:
            raise Exception(f"Initial state format is wrong: {initial_state} but requered (x, y, z, ...) format!")
        if any(amount > size for (amount, size) in zip(initial_state, self._sizes)):
            raise Exception(f"Amounts need to be less or equal the corresponding jug's size!")
        self._initial_state = initial_state
        return self
    
    def set_final_state(self, final_state : tuple[int]):
        if type(final_state) is not tuple[int] and len(final_state) != self._number_of_jugs:
            raise Exception(f"Final state format is wrong: {final_state} but requered (x, y, z, ...) format!")
        if any(amount > size for (amount, size) in zip(final_state, self._sizes)):
            raise Exception(f"Amounts need to be less or equal the corresponding jug's size!")
        self._final_state = final_state
        return self
    
    def fill_jug(self, state : tuple[int, int], which_jug : int = None):
        if which_jug is None:
            new_state = self._sizes
        elif which_jug >= self._number_of_jugs or which_jug < 0:
            raise IndexError(f"Jug {which_jug} not present!")
        else:
            new_jugs = list(state)
            new_jugs[which_jug] = self._sizes[which_jug]
            new_state = tuple(new_jugs)

        return new_state
            
    def empty_jug(self, state : tuple[int, int], which_jug : int = None):
        if which_jug is None:
            new_state = tuple(0 for _ in range(self._number_of_jugs))
        elif which_jug >= self._number_of_jugs or which_jug < 0:
            raise IndexError(f"Jug {which_jug} not present!")
        else:
            new_jugs = list(state)
            new_jugs[which_jug] = 0
            new_state = tuple(new_jugs)

        return new_state
            
    def pour_jug(self, state : tuple[int], from_jug : int, to_jug : int):
        if from_jug == to_jug:
            raise IndexError("from_jug and to_jug cannot be the same!")
        
        if from_jug < 0 and from_jug >= self._number_of_jugs:
            raise IndexError(f"from_jug {from_jug} not allowed")
        
        if to_jug < 0 and to_jug >= self._number_of_jugs:
            raise IndexError(f"to_jug {to_jug} not allowed")

        amount_available_from = state[from_jug]

        if amount_available_from > 0:
            new_jugs = list(state)
            
            amount_available_to = self._sizes[to_jug] - state[to_jug]
            amount_to_pour = amount_available_to if amount_available_from > amount_available_to else amount_available_from
            
            new_jugs[from_jug] = state[from_jug] - amount_to_pour
            new_jugs[to_jug] = state[to_jug] + amount_to_pour

            return tuple(new_jugs)
        else:
            return state

    def generate_combinations_graph(self):
        state_graph = Graph()
        to_visit = [self._initial_state]
        visited = []
        for state in self._all_possible_states:
            state_graph.set_vertex(state)

        while to_visit:
            state = to_visit.pop(0)
            if state not in visited:
                all_reachable_states = []
                all_reachable_states += [self.fill_jug(state, i) for i in range(self._number_of_jugs)]
                all_reachable_states += [self.fill_jug(state)]
                all_reachable_states += [self.empty_jug(state, i) for i in range(self._number_of_jugs)]
                all_reachable_states += [self.empty_jug(state)]
                all_reachable_states += [self.pour_jug(state, i, j) for i in range(self._number_of_jugs) for j in range(self._number_of_jugs) if i != j]

                all_reachable_states = list(set(all_reachable_states))
                new_states = [new_state for new_state in all_reachable_states if new_state in self._all_possible_states and new_state != state]

                to_visit.extend(new_states)
                
                for new_state in new_states:
                    state_graph.add_edge(state, new_state, 1)
                visited.append(state)

        self._state_graph = state_graph

    def solve(self):
        self.generate_combinations_graph()
        all_pairs = all_pairs_shortest_path(self._state_graph)
        return all_pairs[(self._initial_state, self._final_state)]
        

Let's test our code.

In [177]:
jps = JugProblemSolver().set_jugs_number(3).set_jugs_sizes((4, 3, 1)).set_initial_state((0, 0, 0)).set_final_state((2, 3, 0))
jps.solve()

{'distance': 7,
 'route': [((0, 0, 0), (4, 0, 0)),
  ((4, 0, 0), (3, 0, 1)),
  ((3, 0, 1), (3, 1, 0)),
  ((3, 1, 0), (3, 1, 1)),
  ((3, 1, 1), (4, 1, 1)),
  ((4, 1, 1), (2, 3, 1)),
  ((2, 3, 1), (2, 3, 0))]}